In [1]:
import warnings
import logging

root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
import os
import json
import requests
from datetime import datetime
from typing import Dict, Any

# Foursquare API configuration
FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')
BASE_URL = "https://api.foursquare.com"
USER_ID = "34510087"

def test_foursquare_endpoints() -> Dict[str, Any]:
    """Test various Foursquare API endpoints for user check-in data access"""
    findings = {
        "timestamp": datetime.now().isoformat(),
        "endpoints_tested": [],
        "successful_endpoints": [],
        "error_responses": [],
        "recommendations": []
    }
    
    # Headers for API requests
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_API_KEY
    }
    
    # Test endpoints
    endpoints = [
        {
            "name": "User Checkins (v2)",
            "url": f"{BASE_URL}/v2/users/{USER_ID}/checkins",
            "params": {"v": "20240101"}
        },
        {
            "name": "Places API - User History",
            "url": f"{BASE_URL}/v3/places/users/{USER_ID}/history",
            "params": {}
        },
        {
            "name": "Places API - Venue Details",
            "url": f"{BASE_URL}/v3/places/",
            "params": {"user_id": USER_ID}
        }
    ]
    
    for endpoint in endpoints:
        try:
            response = requests.get(
                endpoint["url"],
                headers=headers,
                params=endpoint["params"]
            )
            
            findings["endpoints_tested"].append(endpoint["name"])
            
            if response.status_code == 200:
                findings["successful_endpoints"].append({
                    "name": endpoint["name"],
                    "sample_response": response.json()
                })
            else:
                findings["error_responses"].append({
                    "name": endpoint["name"],
                    "status_code": response.status_code,
                    "error_message": response.text
                })
                
        except Exception as e:
            findings["error_responses"].append({
                "name": endpoint["name"],
                "error": str(e)
            })
    
    # Add recommendations based on findings
    if not findings["successful_endpoints"]:
        findings["recommendations"].extend([
            "User check-in data might require OAuth authentication",
            "Consider using Places API with authorized user token",
            "Alternative: Implement manual check-in feature in the application",
            "Fallback: Use mock data with clear indication to users"
        ])
    
    # Save findings to file
    with open('/data/chats/oky5dd/workspace/docs/foursquare_api_research.md', 'w') as f:
        f.write("# Foursquare API Research Findings\n\n")
        f.write(f"## Test Date: {findings['timestamp']}\n\n")
        
        f.write("## Endpoints Tested\n")
        for endpoint in findings["endpoints_tested"]:
            f.write(f"- {endpoint}\n")
        
        f.write("\n## Successful Endpoints\n")
        for endpoint in findings["successful_endpoints"]:
            f.write(f"### {endpoint['name']}\n")
            f.write("```json\n")
            f.write(json.dumps(endpoint['sample_response'], indent=2))
            f.write("\n```\n\n")
        
        f.write("## Errors Encountered\n")
        for error in findings["error_responses"]:
            f.write(f"### {error['name']}\n")
            f.write(f"```\n{error}\n```\n\n")
        
        f.write("## Recommendations\n")
        for rec in findings["recommendations"]:
            f.write(f"- {rec}\n")
    
    return findings

# Execute the tests and print summary
findings = test_foursquare_endpoints()
print(f"Tests completed. Found {len(findings['successful_endpoints'])} working endpoints.")
print(f"Documented {len(findings['error_responses'])} errors.")
print("Full results saved to foursquare_api_research.md")

Tests completed. Found 0 working endpoints.
Documented 3 errors.
Full results saved to foursquare_api_research.md


In [3]:
import os
import requests
import json
from datetime import datetime
from typing import Dict, Any, Optional

class FoursquareAPI:
    def __init__(self):
        self.base_url = "https://api.foursquare.com/v2"
        self.version = "20240101"
        self.client_id = os.getenv('FOURSQUARE_CLIENT_ID')
        self.client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
        self.redirect_uri = "http://localhost:3000/callback"
        self.user_id = "34510087"
        
    def get_auth_url(self) -> str:
        """Generate OAuth authentication URL"""
        return (f"https://foursquare.com/oauth2/authenticate"
                f"?client_id={self.client_id}"
                f"&response_type=code"
                f"&redirect_uri={self.redirect_uri}")
    
    def get_access_token(self, code: str) -> Optional[str]:
        """Exchange authorization code for access token"""
        url = "https://foursquare.com/oauth2/access_token"
        params = {
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "grant_type": "authorization_code",
            "redirect_uri": self.redirect_uri,
            "code": code
        }
        
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json().get("access_token")
            print(f"Error getting access token: {response.text}")
            return None
        except Exception as e:
            print(f"Exception getting access token: {str(e)}")
            return None
    
    def get_user_checkins(self, access_token: str) -> Dict[str, Any]:
        """Get user's check-in history"""
        url = f"{self.base_url}/users/{self.user_id}/checkins"
        headers = {"Authorization": f"Bearer {access_token}"}
        params = {"v": self.version}
        
        try:
            response = requests.get(url, headers=headers, params=params)
            results = {
                "timestamp": datetime.now().isoformat(),
                "status_code": response.status_code,
                "endpoint": url,
                "success": response.status_code == 200
            }
            
            if response.status_code == 200:
                results["data"] = response.json()
            else:
                results["error"] = response.text
                
            # Log results to file
            with open('/data/chats/oky5dd/workspace/docs/api_test_results.json', 'w') as f:
                json.dump(results, f, indent=2)
                
            return results
            
        except Exception as e:
            error_result = {
                "timestamp": datetime.now().isoformat(),
                "endpoint": url,
                "success": False,
                "error": str(e)
            }
            
            # Log error to file
            with open('/data/chats/oky5dd/workspace/docs/api_test_results.json', 'w') as f:
                json.dump(error_result, f, indent=2)
                
            return error_result

# Initialize API client
api = FoursquareAPI()

# Generate auth URL (first step of OAuth flow)
auth_url = api.get_auth_url()
print("1. Authentication URL (require user to visit):")
print(auth_url)
print("\n2. After user authorization, exchange the 'code' for access token:")
print("api.get_access_token(code='AUTHORIZATION_CODE')")
print("\n3. Use access token to get check-ins:")
print("api.get_user_checkins(access_token='ACCESS_TOKEN')")

# Document findings
with open('/data/chats/oky5dd/workspace/docs/foursquare_api_research.md', 'a') as f:
    f.write("\n\n## OAuth Testing Results\n")
    f.write(f"Test Date: {datetime.now().isoformat()}\n\n")
    f.write("### Authentication URL Generated\n")
    f.write(f"```\n{auth_url}\n```\n\n")
    f.write("### Required Steps\n")
    f.write("1. User must visit the authentication URL\n")
    f.write("2. After authorization, exchange code for access token\n")
    f.write("3. Use access token to access check-in data\n\n")
    f.write("### Notes\n")
    f.write("- OAuth flow requires user interaction\n")
    f.write("- Access tokens should be stored securely\n")
    f.write("- Check-in data access requires user authorization\n")

1. Authentication URL (require user to visit):
https://foursquare.com/oauth2/authenticate?client_id=None&response_type=code&redirect_uri=http://localhost:3000/callback

2. After user authorization, exchange the 'code' for access token:
api.get_access_token(code='AUTHORIZATION_CODE')

3. Use access token to get check-ins:
api.get_user_checkins(access_token='ACCESS_TOKEN')


In [4]:
import os
import requests
import json
from datetime import datetime
from urllib.parse import urlencode

# Set up OAuth credentials
CLIENT_ID = "4V1P22T2MXSSSQBIZVHZBFJCHH1H33KLISIZ1GD0CWRR3F0A"
CLIENT_SECRET = "AHLHJ3QCVZG0H1NK0E3TFADU2MRST2NWXL30YK5EHPY2MAWF"
USER_ID = "34510087"
VERSION = "20240101"
REDIRECT_URI = "http://localhost:3000/callback"

def test_personalization_api():
    """Test Foursquare's Personalization API endpoints with OAuth credentials"""
    
    # Test endpoints to check
    endpoints = [
        {
            "name": "User Details",
            "url": f"https://api.foursquare.com/v2/users/{USER_ID}",
            "method": "GET"
        },
        {
            "name": "User Checkins",
            "url": f"https://api.foursquare.com/v2/users/{USER_ID}/checkins",
            "method": "GET"
        },
        {
            "name": "User Visits",
            "url": f"https://api.foursquare.com/v2/users/{USER_ID}/visits",
            "method": "GET"
        }
    ]
    
    results = {
        "timestamp": datetime.now().isoformat(),
        "client_id": CLIENT_ID,
        "user_id": USER_ID,
        "endpoints_tested": []
    }
    
    # Generate OAuth URL for user authentication
    auth_params = {
        "client_id": CLIENT_ID,
        "response_type": "code",
        "redirect_uri": REDIRECT_URI
    }
    auth_url = f"https://foursquare.com/oauth2/authenticate?{urlencode(auth_params)}"
    results["auth_url"] = auth_url
    
    # Test each endpoint with different auth methods
    for endpoint in endpoints:
        endpoint_result = {
            "name": endpoint["name"],
            "url": endpoint["url"],
            "tests": []
        }
        
        # Test 1: Client ID + Secret
        params = {
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "v": VERSION
        }
        
        try:
            response = requests.get(endpoint["url"], params=params)
            test_result = {
                "auth_method": "client_credentials",
                "status_code": response.status_code,
                "response": response.json() if response.status_code == 200 else response.text
            }
            endpoint_result["tests"].append(test_result)
        except Exception as e:
            endpoint_result["tests"].append({
                "auth_method": "client_credentials",
                "error": str(e)
            })
        
        # Test 2: OAuth token simulation (would require user interaction)
        endpoint_result["tests"].append({
            "auth_method": "oauth",
            "note": "Requires user authentication via OAuth flow",
            "auth_url": auth_url
        })
        
        results["endpoints_tested"].append(endpoint_result)
    
    # Save results to file
    with open('/data/chats/oky5dd/workspace/docs/api_test_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    # Update research document
    with open('/data/chats/oky5dd/workspace/docs/foursquare_api_research.md', 'a') as f:
        f.write("\n\n## API Test Results - " + datetime.now().isoformat())
        f.write("\n\nTested with provided OAuth credentials:\n")
        f.write(f"- Client ID: {CLIENT_ID}\n")
        f.write("- User ID: 34510087\n\n")
        f.write("### Authentication URL\n")
        f.write(f"```\n{auth_url}\n```\n\n")
        f.write("### Endpoints Tested\n")
        for endpoint in results["endpoints_tested"]:
            f.write(f"\n#### {endpoint['name']}\n")
            f.write(f"- URL: {endpoint['url']}\n")
            f.write("- Test Results:\n")
            for test in endpoint["tests"]:
                f.write(f"  - Auth Method: {test['auth_method']}\n")
                if "status_code" in test:
                    f.write(f"  - Status Code: {test['status_code']}\n")
                if "note" in test:
                    f.write(f"  - Note: {test['note']}\n")
                f.write("\n")

# Run the tests
test_personalization_api()
print("API tests completed. Results saved to api_test_results.json and foursquare_api_research.md")

API tests completed. Results saved to api_test_results.json and foursquare_api_research.md
